<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg">
## Open Machine Learning Course
<center>Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/), Data Scientist at Mail.ru Group <br>
    All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.
You may use this material for any purpose (you can edit, correct and use it as example) exept commercial use with mandatory citation of author.

# <center> Assignment #6 (demo).
## <center>  Exploring OLS, Lasso and Random Forest in a regression task
    
<img src=https://habrastorage.org/webt/-h/ns/aa/-hnsaaifymavmmudwip9imcmk58.jpeg width=30%>

**Fill in the missing code and choose answers in [this](https://docs.google.com/forms/d/1aHyK58W6oQmNaqEfvpLTpo6Cb0-ntnvJ18rZcvclkvw/edit) web form.**

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**We are working with UCI Wine quality dataset (no need to download it – it's already there, in course repo and in Kaggle Dataset).**

In [2]:
data = pd.read_csv('../input/winequality-white.csv')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with `StandardScaler`.**

In [5]:
y = data['quality']

X_train, X_holdout, y_train, y_holdout = train_test_split(data.drop(columns='quality'), y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Linear regression

**Train a simple linear regression model (Ordinary Least Squares).**

In [6]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

**<font color='red'>Question 1:</font> What are mean squared errors of model predictions on train and holdout sets?**

In [16]:
print("Mean squared error (train): %.3f" % (mean_squared_error(y_train, linreg.predict(X_train_scaled))))
print("Mean squared error (test): %.3f" % (mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled))))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It's handy to use `pandas.DataFrame` here.**

**<font color='red'>Question 2:</font> Which feature this linear regression model treats as the most influential on wine quality?**

In [36]:
linreg_coef = pd.DataFrame(data=np.abs(linreg.coef_), index=X_train.columns, columns=['Сoefficients'])
linreg_coef.sort_values(by='Сoefficients', ascending=False)

,Сoefficients
density,0.665720
residual sugar,0.538164
volatile acidity,0.192260
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127


## Lasso regression

**Train a LASSO model with $\alpha = 0.01$ (weak regularization) and scaled data. Again, set random_state=17.**

In [32]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [37]:
lasso1_coef = pd.DataFrame(data=np.abs(lasso1.coef_), index=X_train.columns, columns=['Сoefficients'])
lasso1_coef.sort_values(by='Сoefficients')

,Сoefficients
fixed acidity,0.000000
citric acid,0.000000
total sulfur dioxide,0.000000
chlorides,0.002747
sulphates,0.029722
free sulfur dioxide,0.043088
pH,0.067277
volatile acidity,0.188479
density,0.235492
residual sugar,0.256363


**Train LassoCV with random_state=17 to choose the best value of $\alpha$ in 5-fold cross-validation.**

In [38]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, random_state=17, n_jobs=-1, cv=5)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=-1, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [39]:
lasso_cv.alpha_

0.00023542864143224178

**<font color='red'>Question 3:</font> Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [41]:
lasso_cv_coef = pd.DataFrame(data=np.abs(lasso_cv.coef_), index=X_train.columns, columns=['Сoefficients'])
lasso_cv_coef.sort_values(by='Сoefficients')

,Сoefficients
citric acid,0.000000
chlorides,0.007131
total sulfur dioxide,0.013193
free sulfur dioxide,0.042610
sulphates,0.061126
fixed acidity,0.094054
alcohol,0.135826
pH,0.147142
volatile acidity,0.192079
residual sugar,0.528794


**<font color='red'>Question 4:</font> What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [43]:
print("Mean squared error (train): %.3f" % (mean_squared_error(y_train, lasso_cv.predict(X_train_scaled))))
print("Mean squared error (test): %.3f" % (mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled))))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Random Forest

**Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.**

In [61]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

**<font color='red'>Question 5:</font> What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [62]:
cv_score = cross_val_score(estimator=forest, X=X_train, y=y_train, scoring='neg_mean_squared_error').mean()

print("Mean squared error (train): %.3f" % (mean_squared_error(y_train, forest.predict(X_train))))
print("Mean squared error (cv): %.3f" % (cv_score))
print("Mean squared error (test): %.3f" % (mean_squared_error(y_holdout, forest.predict(X_holdout))))

Mean squared error (train): 0.076
Mean squared error (cv): -0.462
Mean squared error (test): 0.422


**Tune the `max_features` and `max_depth` hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [50]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(estimator=forest, param_grid=forest_params)
locally_best_forest.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7], 'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [51]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 19, 'max_features': 6, 'min_samples_leaf': 1},
 0.42144574416375985)

**<font color='red'>Question 6:</font> What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [55]:
best_forest = RandomForestRegressor(**locally_best_forest.best_params_, random_state=17)
best_forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=None, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [56]:
print("Mean sqRandomForestRegressorred error (cv): %.3f" % (cross_val_score(estimator=best_forest, X=X_train, y=y_train, scoring='neg_mean_squared_error').mean()))
print("Mean squared error (test): %.3f" % (mean_squared_error(y_holdout, best_forest.predict(X_holdout))))

Mean sqRandomForestRegressorred error (cv): -0.455
Mean squared error (test): 0.412


**Output RF's feature importance. Again, it's nice to present it as a DataFrame.**<br>
**<font color='red'>Question 7:</font> What is the most important feature, according to the Random Forest model?**

In [58]:
rf_importance = pd.DataFrame(data=np.abs(best_forest.feature_importances_), index=X_train.columns, columns=['Сoefficients'])
rf_importance.sort_values(by='Сoefficients', ascending=False)

,Сoefficients
alcohol,0.212597
volatile acidity,0.118316
free sulfur dioxide,0.114179
chlorides,0.084255
density,0.075461
pH,0.074953
residual sugar,0.072263
total sulfur dioxide,0.072215
fixed acidity,0.060752
sulphates,0.058372


**Make conclusions about the perdormance of the explored 3 models in this particular prediction task.**